Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "05")
cd(ProjDir)

### snippet 5.1

In [3]:
wd = CSV.read(rel_path("..", "data", "WaffleDivorce.csv"), delim=';')
df = convert(DataFrame, wd);

mean_ma = mean(df[:Marriage])
df[:Marriage_s] = convert(Vector{Float64},
  (df[:Marriage]) .- mean_ma)/std(df[:Marriage]);

mean_mam = mean(df[:MedianAgeMarriage])
df[:MedianAgeMarriage_s] = convert(Vector{Float64},
  (df[:MedianAgeMarriage]) .- mean_mam)/std(df[:MedianAgeMarriage]);

df[1:6, [1, 7, 14, 15]]

rethinking_data = "
    Location Divorce  Marriage.s MedianAgeMarriage.s
1    Alabama    12.7  0.02264406          -0.6062895
2     Alaska    12.5  1.54980162          -0.6866993
3    Arizona    10.8  0.04897436          -0.2042408
4   Arkansas    13.5  1.65512283          -1.4103870
5 California     8.0 -0.26698927           0.5998567
"

"\n    Location Divorce  Marriage.s MedianAgeMarriage.s\n1    Alabama    12.7  0.02264406          -0.6062895\n2     Alaska    12.5  1.54980162          -0.6866993\n3    Arizona    10.8  0.04897436          -0.2042408\n4   Arkansas    13.5  1.65512283          -1.4103870\n5 California     8.0 -0.26698927           0.5998567\n"

Define the Stan language model

In [4]:
m5_3_model = "
data {
  int N;
  vector[N] divorce;
  vector[N] marriage_z;
  vector[N] median_age_z;
}
parameters {
  real a;
  real bA;
  real bM;
  real<lower=0> sigma;
}
model {
  vector[N] mu = a + median_age_z * bA + marriage_z * bM;
  sigma ~ uniform( 0 , 10 );
  bA ~ normal( 0 , 1 );
  bM ~ normal( 0 , 1 );
  a ~ normal( 10 , 10 );
  divorce ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m5_3_model",
monitors = ["a", "bA", "bM", "sigma", "Divorce"],
 model=m5_3_model, output_format=:mcmcchain);
# Input data for cmdstan
m5_3_data = Dict("N" => size(df, 1), "divorce" => df[:Divorce],
    "marriage_z" => df[:Marriage_s], "median_age_z" => df[:MedianAgeMarriage_s]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, m5_3_data, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)


Inference for Stan model: m5_3_model_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.095, 0.12, 0.085, 0.11) seconds, 0.41 seconds total
Sampling took (0.10, 0.097, 0.10, 0.10) seconds, 0.40 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat
lp__              -46  3.7e-02     1.5    -48    -45    -44  1.6e+03  4.0e+03  1.0e+00
accept_stat__    0.91  1.5e-03    0.11   0.69   0.95    1.0  5.0e+03  1.2e+04  1.0e+00
stepsize__       0.56  1.6e-02   0.023   0.53   0.56   0.59  2.0e+00  5.0e+00  2.8e+13
treedepth__       2.6  8.7e-03    0.52    2.0    3.0    3.0  3.7e+03  9.2e+03  1.0e+00
n_leapfrog__      6.6  4.5e-02     2.8    3.0    7.0     15  3.9e+03  9.7e+03  1.0e+00
divergent__      0.00      nan    0.00   0.00   0.00   0.00      nan      nan      nan
energy__           48  5.4e-02     2.0     45     47     51  1.4e+03  3.4e+03  1.0e+00
a                 

Rethinking results

In [6]:
rethinking_results = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.69 0.22  9.34 10.03  1313    1
bR    -0.12 0.30 -0.60  0.35   932    1
bA    -1.13 0.29 -1.56 -0.67   994    1
sigma  1.53 0.16  1.28  1.80  1121    1
"

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.69 0.22  9.34 10.03  1313    1\nbR    -0.12 0.30 -0.60  0.35   932    1\nbA    -1.13 0.29 -1.56 -0.67   994    1\nsigma  1.53 0.16  1.28  1.80  1121    1\n"

End of `05/5.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*